# Machine learning and statistical learning

## Pictures: Parkinson’s Disease

This hands-on session focuses on pictures. The objective is to manipulate images to perform classification tasks.

The task consists in detecting Parkinson's disease from hand-drawn images of spirals and waves. This application is inspired by [Adrian Rosebrock's tutorial](https://www.pyimagesearch.com/2019/04/29/detecting-parkinsons-disease-with-opencv-computer-vision-and-the-spiral-wave-test/). 


The idea is to compare spiral or wave images drawn by patients with Parkinson's disease or by patients without it. Patients with Parkinson's disease, as shown in [Zham _et al_. (2017)](https://doi.org/10.3389%2Ffneur.2017.00435), draw less quickly and with less pressure than other test subjects. In the tutorial, Adrian Rosebrock mentions that João Paulo Folador, a Brazilian Ph.D. student explored the following idea: maybe it is possible to detect Parkinson's disease from the sole images, without measuring speed and pressure of the pen on paper.

you will **train a classifier** to automatically detect whether a patient is suffering from Parkinson's disease.

To that end, you have a set of image data organized as follows:

- Spiral data:

    - training set: 36 healthy and 36 Parkinson (n=72)
    - testing set: 15 healthy and 15 Parkinson (n=30)
- Wave data:

    - training set: 36 healthy and 36 Parkinson (n=72)
    - testing set: 15 healthy and 15 Parkinson (n=30).

You will adopt the following strategy:

- load the images into python
- convert them in greyscale
- crop them
- create a matrix where each column represents an example (i.e., an individual) and each line the value of a pixel
- perform a principal component analysis
- use a subset of the principal components to train a classifier
- test the performance of the model on a test sample

## Data Preparation

### Loading images

The hand-drawings of the subjects are provided as PNG files, according to the following tree structure (you will focus on the spiral test only):

```
pictures
├── donnees
│   └── dataset-parkinson
│       └── spiral
│           └── testing
│           │   └── healthy
│           │   │   └── V01HE01.png
│           │   │   └── ...
│           │   │   └── V55HE15.png
│           │   └── parkinson
│           │   │   └── V01PE01.png
│           │   │   └── ...
│           │   │   └── V15PE015.png
│           └── training
│           │   └── healthy
│           │   │   └── V01HE02.png
│           │   │   └── ...
│           │   │   └── V55HE11.png
│           │   └── parkinson
│           │   │   └── V01PE02.png
│           │   │   └── ...
│           │   │   └── V15PE03.png
```

The file names can be used to determine whether or not the individual has Parkinson's disease (P for Parkinson's or H for Healthy). The data are already split into a training and a testing sample.

Let us list all the files, using the `listdir()` function from {`os`}.

In [1]:
import os

In [2]:
os.getcwd()

'/Users/lilyhuong/Desktop/Amse mag3/Machine learning statistics'

In [110]:
import os
path_train_healthy = "/Users/lilyhuong/Desktop/Amse mag3/Machine learning statistics/dataset/spiral/training/healthy"
N_train_healthy = os.listdir(path_train_healthy)

path_train_parkinson = "/Users/lilyhuong/Desktop/Amse mag3/Machine learning statistics/dataset/spiral/training/parkinson"
N_train_parkinson = os.listdir(path_train_parkinson)

path_test_healthy = "/Users/lilyhuong/Desktop/Amse mag3/Machine learning statistics/dataset/spiral/testing/healthy"
N_test_healthy = os.listdir(path_test_healthy)

path_test_parkinson = "/Users/lilyhuong/Desktop/Amse mag3/Machine learning statistics/dataset/spiral/testing/parkinson"
N_test_parkinson = os.listdir(path_test_parkinson)

In [111]:
N = [[path_train_healthy,   N_train_healthy],
     [path_train_parkinson, N_train_parkinson],
     [path_test_healthy,    N_test_healthy],
     [path_test_parkinson,  N_test_parkinson]
]

In [13]:
N

[['/Users/lilyhuong/Desktop/Amse mag3/Machine learning statistics/dataset-parkinson/spiral/training/healthy',
  ['V09HE02.png',
   'V55HE06.png',
   'V55HE07.png',
   'V09HE03.png',
   'V55HE11.png',
   'V55HE05.png',
   'V10HE02.png',
   'V10HE03.png',
   'V55HE04.png',
   'V55HE10.png',
   'V01HE02.png',
   'V11HE02.png',
   'V11HE03.png',
   'V01HE03.png',
   'V55HE01.png',
   'V08HE02.png',
   'V55HE03.png',
   'V55HE02.png',
   'V08HE03.png',
   'V12HE01.png',
   'V02HE02.png',
   'V12HE02.png',
   'V12HE03.png',
   'V02HE03.png',
   'V07HE03.png',
   'V07HE02.png',
   'V03HE2.png',
   'V06HE03.png',
   'V06HE02.png',
   'V03HE3.png',
   'V04HE03.png',
   'V04HE02.png',
   'V05HE03.png',
   'V55HE09.png',
   'V55HE08.png',
   'V05HE02.png']],
 ['/Users/lilyhuong/Desktop/Amse mag3/Machine learning statistics/dataset-parkinson/spiral/training/parkinson',
  ['V09PE03.png',
   'V10PE01.png',
   'V09PE02.png',
   'V10PE02.png',
   'V01PE03.png',
   'V01PE02.png',
   'V11PE02.png',
   '

You need to load the images. In a first step, let us load only **a single image** and prepare it. Then you can make a loop to deal with all the images.

In [20]:
path_img = N[0][0] + "/" + N[0][1][0]

You can use the `imread()` function from [OpenCV](https://pypi.org/project/opencv-python/) to process the image.

In [30]:
!pip install opencv-python

     |████████████████████████████████| 46.4 MB 846 kB/s eta 0:00:01


In [31]:
import cv2
import numpy as np

In [32]:
from skimage.io import imread

In [33]:
image = imread(path_img)

The image is loaded as a numpy ndarray. It has 3 dimensions:

- width
- height
- colour: Red, Green, and Blue (RGB) components

In [22]:
type(image)

numpy.ndarray

Print the shape of the image:

In [23]:
image.shape

(256, 256, 3)

You will then perform a principal component analysis (PCA) on the pixel values of each image. The main components will then be used as explanatory variables (features) to train a binary classifier to detect whether the drawings come from a healthy person or a person with Parkison’s disease. But you are facing a problem at this stage. PCA is an orthogonal linear transformation, and the data consists of triplet for each pixel, indicating the red, green and blue components (RGB). A simple way to get around this problem is to convert the image to grey scale, _i.e._, by calculating the average of the three RGB components. This is what the `cvtColor()` function from {cv2} does.

In [34]:
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [35]:
image_gray.shape

(256, 256)

The images in this dataset are not all exactly the same size. Most of them are squares of 256 x 256 pixels. For the few exceptions, the difference is minimal. But for those, you need to resise the images.

If interpolation is needed, you can use the nearest neighbours.

Resize the image to 256 pixels x 256 pixels

In [38]:
image_resized = cv2.resize(image_gray, (256, 256), interpolation = cv2.INTER_NEAREST)

In [39]:
image_resized

array([[240, 239, 241, ..., 240, 244, 245],
       [246, 243, 245, ..., 243, 235, 238],
       [240, 244, 250, ..., 238, 236, 237],
       ...,
       [247, 234, 232, ..., 236, 237, 236],
       [247, 237, 232, ..., 240, 231, 245],
       [249, 242, 233, ..., 243, 235, 242]], dtype=uint8)

Let us have a look at our example.

Plot your image.

Now, create a loop over all files and apply the same pre-processing.

For each example that you load, you must know whether it is from the training or the test sample (you can store this information on an array that you can call `sample`), and you must also know the true value (`healthy` or `parkinson`). To do so, from the path to the example that is being loaded, you can use the `seatch()` function from the `re` library and look for some pattern within the string.



In [40]:
import re

In [53]:
type_sample_current = []


In [109]:
path_folder

'/Users/lilyhuong/Desktop/Amse mag3/Machine learning statistics/dataset-parkinson/spiral/testing/parkinson'

In [52]:
examples = []
type_example = []
sample = []
for i_folder in range(len(N)):
    path_folder = N[i_folder][0]
    for i_image in range(len((N[i_folder][1]))):
        file_img = N[i_folder][1][i_image]
        path_img = path_folder + '/'+ file_img
        image = cv2.imread(path_img)
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image_resized = cv2.resize(image_gray, (256, 256), interpolation = cv2.INTER_NEAREST)
        examples.append(image_resized)
        
        if bool(re.search("parkinson", path_folder)):
            type_example.append("parkinson")
        else:
            type_example.appe,d("healthy")

        if bool(re.search("training", path_folder)):
            sample.append("training")
        else:
            sample.append("test")


In [73]:
examples = []
type_example = []
sample = []
for i_folder in range(len(N)):
    path_folder = N[i_folder][0]
    for i_image in range(len(N[i_folder][1])):
        file_img = N[i_folder][1][i_image]
        path_img = path_folder+"/"+file_img
        # Load the image
        image = cv2.imread(path_img)
        # Grey scale
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY )
        # Resize
        image_resized = cv2.resize(image_gray, (256, 256), interpolation = cv2.INTER_NEAREST)
        # Add the resized image to our train or test data
        #examples.append(image_resized)
        examples.append(image_resized.flatten())
        # True answer: is it healty or parkinson?
        if bool(re.search("parkinson", path_folder)):
            type_example.append(1)
        else:
            type_example.append(0)
        # Type of sample: train or test
        if bool(re.search("training", path_folder)):
            sample.append("training")
        else:
            sample.append("testing")



In [108]:
type_example

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [74]:
examples

[array([240, 239, 241, ..., 243, 235, 242], dtype=uint8),
 array([244, 235, 240, ..., 244, 240, 238], dtype=uint8),
 array([242, 235, 235, ..., 234, 236, 232], dtype=uint8),
 array([242, 244, 241, ..., 238, 239, 238], dtype=uint8),
 array([238, 238, 246, ..., 236, 236, 239], dtype=uint8),
 array([235, 232, 233, ..., 236, 240, 239], dtype=uint8),
 array([240, 240, 227, ..., 236, 241, 240], dtype=uint8),
 array([230, 243, 243, ..., 243, 234, 233], dtype=uint8),
 array([237, 238, 236, ..., 230, 232, 232], dtype=uint8),
 array([234, 234, 231, ..., 218, 224, 235], dtype=uint8),
 array([243, 242, 243, ..., 227, 227, 234], dtype=uint8),
 array([237, 238, 239, ..., 241, 238, 243], dtype=uint8),
 array([231, 212, 253, ..., 246, 234, 236], dtype=uint8),
 array([233, 235, 227, ..., 232, 245, 238], dtype=uint8),
 array([237, 237, 239, ..., 230, 231, 235], dtype=uint8),
 array([242, 240, 239, ..., 237, 233, 242], dtype=uint8),
 array([241, 239, 237, ..., 236, 240, 236], dtype=uint8),
 array([238, 2

### Rescaling the data

Before using PCA, you need to rescale the data so that the range of the values lie in the [0,1] interval. To that end, you can use a minmax scaler.

$x_{i, \text{scaled}} = \frac{x_i-min(x_i)}{max(x_i) - min(x_i)}$


In [65]:
from sklearn.preprocessing import MinMaxScaler

In [66]:
scaler = MinMaxScaler()

In [75]:
examples_scaled = scaler.fit_transform(examples)

In [76]:
examples_scaled

array([[0.69767442, 0.65853659, 0.61764706, ..., 0.68421053, 0.42857143,
        0.64705882],
       [0.79069767, 0.56097561, 0.58823529, ..., 0.71052632, 0.60714286,
        0.52941176],
       [0.74418605, 0.56097561, 0.44117647, ..., 0.44736842, 0.46428571,
        0.35294118],
       ...,
       [0.90697674, 0.85365854, 0.82352941, ..., 0.76315789, 0.75      ,
        0.47058824],
       [0.86046512, 0.6097561 , 0.47058824, ..., 0.78947368, 1.        ,
        0.76470588],
       [0.51162791, 0.65853659, 0.64705882, ..., 0.26315789, 0.17857143,
        0.61764706]])

Now, create your training and testing datasets:

In [77]:
mask_training = np.array(sample) == "training"
y_train = np.array(type_example)[mask_training]
y_test = np.array(type_example)[np.logical_not(mask_training)]

X_train = np.array(examples_scaled)[mask_training]
X_test = y_test = np.array(examples_scaled)[np.logical_not(mask_training)]

### Summary

In [78]:
(y_train.shape, X_train.shape)

((72,), (72, 65536))

In [79]:
(y_test.shape, X_test.shape)

((30, 65536), (30, 65536))

## PCA

As previously mentioned, you can perform a principal component analysis on the pixel values to extract the first components that can then be used to train the classifier. To do so, you give the matrix containing our training observations to the `PCA()` function from the module `decomposition` of the library `scikit-learn`.

In [80]:
from sklearn.decomposition import PCA

Assuma that you want to extract only the first three components:

In [81]:
M = 3

Fit the model on `X_train`

The components are returned in an array:

In [86]:
pca = PCA(n_components= M)

In [87]:
pca.fit(X_train)

PCA(n_components=3)

Now, you can apply the dimensionality reduction on the data, using the `transform()` function from `pca`.

In [88]:
pca.components_

array([[-1.55923334e-03,  5.22367370e-04,  4.77874173e-05, ...,
        -9.27882410e-04,  7.95349540e-04,  2.64870074e-03],
       [ 2.52142737e-03,  6.08967567e-03,  6.40812905e-03, ...,
         1.33130338e-03,  4.70842376e-03,  8.11910280e-03],
       [ 6.29549551e-03,  6.15010792e-03,  4.09731614e-03, ...,
         3.29736046e-03,  1.64773618e-03,  4.80304806e-03]])

In [90]:
X_train_reduced = pca.transform(X_train)
X_test_reduced =  pca.transform(X_test)

In [107]:
X_train_reduced

array([[-9.08612302e+00,  4.53233419e+00,  7.96695841e+00],
       [-3.67470562e+00,  1.12870077e+00,  1.22472602e+00],
       [ 4.52382859e+00, -1.11545024e+00, -7.26825657e+00],
       [-8.79092874e+00,  4.10209341e+00,  5.73817688e+00],
       [-1.00743634e+00, -1.74901167e+00, -3.45302666e+00],
       [-1.06123612e+00, -2.38362932e+00, -1.99570772e+00],
       [-1.15986619e+00, -4.14112613e-01,  5.56216646e-01],
       [ 1.38692211e+00, -3.61344056e+00, -3.53131628e+00],
       [-2.78180228e+00,  1.26059404e-01,  5.21861957e-01],
       [-9.05726722e-01, -7.17666391e+00,  3.02315706e+00],
       [ 3.13035926e-01, -4.21422391e+00,  1.14328968e+01],
       [ 3.51337227e+00,  2.42057891e+00, -4.68935532e+00],
       [-3.38117008e+00,  6.11744684e-01, -2.32919980e-01],
       [ 8.36250022e+00, -5.28616924e+00, -1.52789683e+01],
       [-9.79849330e-01, -3.19943152e+00, -3.10419628e+00],
       [-2.57747617e-01, -9.89891305e-01,  1.35101000e+00],
       [-1.15143798e+00, -1.60134860e-01

In [91]:
print("Dimension of the reduced training set: ", X_train_reduced.shape)
print("Dimension of the reduced testing set: ", X_test_reduced.shape)

Dimension of the reduced training set:  (72, 3)
Dimension of the reduced testing set:  (30, 3)


## Model Training


In this section you will train a classifier to recognize from the drawings of the subjects whether or not they have Parkinson's disease. To start slowly, you will put in place an example in which you will only retain the first three main components from the PCA that was conducted previously. The strategy you will use is as follows:

- create the data table that will be used to train the classifier
- train the classifier with a logistic regression (`LogisticRegression()`)
- measure the performance of the model.

In [97]:
from sklearn.linear_model import LogisticRegression

In [106]:
reg_logit = LogisticRegression(random_state=0).fit(X_train_reduced, y_train)

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1

The predicted values for both samples:

In [101]:
y_pred_train = reg_logit(X_train)
y_pred_test = reg_logit(X_test)

TypeError: 'LogisticRegression' object is not callable

In [ ]:
from sklearn.metrics import confusion_matrix

The confusion matrix for the training sample:

- in rows: true label
- in columns: predicted class

And for the test sample:

You can also use the `classification_report()` function from `classification_report`.

In [ ]:
from sklearn.metrics import classification_report

- **precision**: ability of the classifier not to label as positive a sample that is negative $\frac{TP}{TP+FP}$
- **recall**: ability of the classifier to find all the positive samples, true positive rate $\frac{TP}{TP+FN}$
- **f1-score**: harmonic mean of precision and recall

On the training sample:

And on the test sample:

## Loop over the number of components to use

The higher the number of principal components used, the closer the projected data are from the initial dataset. Create a loop over M to try to know how many components seem to be enough to represent correctly the data.

You can select M depending on the percentage of the cumulative variance that is explained.

But fitst, go back to your example with M=3 components kept.

In [ ]:
M = 3

In [ ]:
pca = PCA(n_components=M)

In [ ]:
pca.fit(X_train)

The variance explained by each dimension is returned in the `explained_variance_ratio_` element:

In [ ]:
pca.explained_variance_ratio_

To know the cumulative variance explained by the first dimensions:

In [ ]:
pca.explained_variance_ratio_.cumsum()

Set the value of the number of components to 70:

In [ ]:
M=70

Perform the PCA with 70 components.

From the attribute `explained_variance_ratio_` in your model objetc, compute the cumulative variance explained by the first dimensions

In [ ]:
cumsum_variance = 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(range(1,M+1), cumsum_variance*100)
plt.title("Percentage of variance explained as a function of the number of dimensions used")
plt.xlabel("Number of principal components")

Retrieve (using a code, not setting the value by hand) the number of components needed so that the variance explained by the dimensions is at least 90%.

In [ ]:
M = 

Alternatively, you can specify to the argument `n_components` `PCA()` the deisired percentage of explained variance you wish. The number of components kept is then based on that.

In [ ]:
# pca = PCA(n_components = 0.80)

Perform CPA with this number of components.

In [ ]:
pca = 

Using the `transform()` method, apply the reduction to your train and test sets.

In [ ]:
X_train_reduced = 
X_test_reduced = 

Then, fit a logistic model on the reduced train data to predict whether the patient has the parkinson disease or not.

In [ ]:
reg_logit = 

Use your estimated model to get the predictions on the train and on the test sets.

In [ ]:
y_pred_train = 
y_pred_test = 

Show the confusion matrix on the train set.

In [ ]:
cf_matrix_train = 
cf_matrix_train

As a bonus, some code to visualize this matrix...

In [ ]:
import seaborn as sns

In [ ]:
# Code from https://medium.com/@dtuk81/confusion-matrix-visualization-fc31e3f30fea
cf_matrix = cf_matrix_train
cf_names = ["True Neg.", "False Pos.", "False Neg.", "True Pos."]
cf_n = ["{0:0.0f}".format(val) for val in cf_matrix.flatten()]
cf_pct = ["{0:.2%}".format(val) for val in cf_matrix.flatten() / np.sum(cf_matrix)]
cf_labels = [f"{val_1}\n{val_2}\n{val_3}" for val_1, val_2, val_3 in zip(cf_names,cf_n,cf_pct)]
cf_labels = np.asarray(cf_labels).reshape(2,2)

ax = sns.heatmap(cf_matrix, annot=cf_labels, 
            fmt='', cmap='Blues')
ax.set_title("Confusion matrix, training test")

And now, on the test set:

In [ ]:
cf_matrix_test = 
cf_matrix_test

In [ ]:
cf_matrix = cf_matrix_test
cf_names = ["True Neg.", "False Pos.", "False Neg.", "True Pos."]
cf_n = ["{0:0.0f}".format(val) for val in cf_matrix.flatten()]
cf_pct = ["{0:.2%}".format(val) for val in cf_matrix.flatten() / np.sum(cf_matrix)]
cf_labels = [f"{val_1}\n{val_2}\n{val_3}" for val_1, val_2, val_3 in zip(cf_names,cf_n,cf_pct)]
cf_labels = np.asarray(cf_labels).reshape(2,2)

ax = sns.heatmap(cf_matrix, annot=cf_labels, 
            fmt='', cmap='Blues')
ax.set_title("Confusion matrix, testing test")